## Setup: Install Dependencies

In [ ]:
# Dependencies - verify they are installed
!pip install --disable-pip-version-check --quiet -U langchain==0.2.16
!pip install --disable-pip-version-check --quiet -U langchain_openai==0.1.23
!pip install --disable-pip-version-check --quiet -U langgraph==0.2.19
!pip install --disable-pip-version-check --quiet -U langchainhub==0.1.21
!pip install --disable-pip-version-check --quiet -U tavily-python==0.4.0
!pip install --disable-pip-version-check --quiet -U langchain-community==0.2.16
!pip install --disable-pip-version-check --quiet -U python-dotenv==1.0.1

print("All dependencies installed successfully!")

## Load Environment Variables

In [ ]:
from dotenv import load_dotenv
import os

# Load from root .env (workspace root)
# Navigate from notebooks/ -> healthbot/ -> project/ -> root
root_env = os.path.normpath(os.path.join(os.getcwd(), '..', '..', '..', '.env'))
print(f"Loading environment from: {root_env}")
print(f"File exists: {os.path.exists(root_env)}")

load_dotenv(root_env)

# Verify Azure Foundry credentials
assert os.getenv('FOUNDRY_PROJECT_ENDPOINT') is not None, "Missing FOUNDRY_PROJECT_ENDPOINT"
assert os.getenv('FOUNDRY_API_KEY') is not None, "Missing FOUNDRY_API_KEY"
assert os.getenv('FOUNDRY_DEPLOYMENT_NAME') is not None, "Missing FOUNDRY_DEPLOYMENT_NAME"

# Verify Tavily API key
assert os.getenv('TAVILY_API_KEY') is not None, "Missing TAVILY_API_KEY"

print("All environment variables loaded successfully!")

## Import HealthBot Modules

In [ ]:
import sys
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

from workflow import create_healthbot_workflow, create_config, initialize_empty_state
from llm_config import initialize_llm

print("HealthBot modules imported successfully!")

## Initialize Workflow

In [ ]:
# Create the LangGraph workflow
app = create_healthbot_workflow()

print("HealthBot workflow created successfully!")

## Display Workflow Diagram

In [ ]:
from IPython.display import Image, display

try:
    graph_image = app.get_graph().draw_mermaid_png()
    print("HealthBot Workflow Diagram:")
    display(Image(graph_image))
except Exception as e:
    print(f"Could not display workflow diagram: {e}")
    print("This is optional - workflow will still run fine.")

## Run HealthBot

Below you will have an interactive session with HealthBot. Follow the prompts to:
1. Enter a health topic you want to learn about
2. Read the medical summary
3. Answer the quiz question
4. See your grade and feedback
5. Choose: more questions on this topic, learn a new topic, or exit

In [ ]:
# Create configuration
config = create_config(thread_id="healthbot_session_main")

# Initialize empty state
initial_state = initialize_empty_state()

print("\n" + "="*80)
print("STARTING HEALTHBOT SESSION")
print("="*80 + "\n")

try:
    # Run the workflow
    final_state = app.invoke(initial_state, config)
    
    print("\n" + "="*80)
    print("SESSION COMPLETE")
    print("="*80)
    print(f"\nFinal state captured. Messages exchanged: {len(final_state['messages'])}")
    
except Exception as e:
    print(f"\nError during HealthBot execution: {str(e)}")
    import traceback
    traceback.print_exc()

## Session Summary

In [ ]:
print("\n" + "="*80)
print("SESSION SUMMARY")
print("="*80)

if 'final_state' in locals():
    print(f"\nTopics discussed: {final_state.get('health_topic', 'None')}")
    print(f"Total messages: {len(final_state.get('messages', []))}")
    print(f"Final grade: {final_state.get('grade', 'N/A')}/100")
    print(f"Session ended: {'User requested exit' if final_state.get('should_continue') != 'more_questions' else 'Continued'}")
else:
    print("No session data available - check errors above.")

## Test the Stand-Out Feature (Optional)

To test the multiple quiz questions feature:
1. Run the workflow above
2. When asked what to do next, choose option (1) "Another quiz question on this topic"
3. The question should be different from the first one but still answerable from the same summary
4. This demonstrates the stand-out feature that allows deeper learning without restarting